# 04. Feature Engineering (파생변수 생성)

---

## 목표
1. 수정주가 데이터 로드 (`stock_daily_master_final.csv`)
2. 데이터 품질 검증 (QA)
3. 파생변수 계산 (수익률, 리스크, 거래량, 기술지표)
4. 결측치 처리 및 Clean 데이터 저장

## 파생변수 설계 원칙
- **Daily_Return 첫 값은 NaN**: 전일 가격이 없어 정의 불가
- **누적수익률(Cum_Return)**: 계산용으로 첫 값을 0으로 대체한 별도 변수 사용
- **이동평균/변동성**: `min_periods=window`로 설정하여 초기 구간은 NaN 유지
- **거래량**: log1p + winsorize(1%) 적용하여 분포 정규화

---

## Section 1. 환경 설정 & 데이터 로드

In [1]:
# Library Imports
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# 시각화 설정
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("라이브러리 로드 완료")

라이브러리 로드 완료


In [2]:
# 경로 설정
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / 'Data_set'
QA_DIR = DATA_DIR / 'QA'

QA_DIR.mkdir(exist_ok=True)

print(f"프로젝트 경로: {PROJECT_ROOT}")
print(f"데이터 경로: {DATA_DIR}")

프로젝트 경로: /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance
데이터 경로: /Users/yu_seok/Documents/workspace/nbCamp/Project/Yahoo Finance/Data_set


In [3]:
# 데이터 로드
print("=" * 60)
print("데이터 로드")
print("=" * 60)

input_file = DATA_DIR / 'stock_daily_master.csv'

if not input_file.exists():
    raise FileNotFoundError(f"파일이 없습니다: {input_file}\n03_Data_Update.ipynb를 먼저 실행하세요.")

df = pd.read_csv(input_file)
df['Date'] = pd.to_datetime(df['Date'])

print(f"\n로드 완료")
print(f"  Shape: {df.shape}")
print(f"  기간: {df['Date'].min().date()} ~ {df['Date'].max().date()}")
print(f"  기업 수: {df['Company'].nunique()}")
print(f"\n컬럼 ({len(df.columns)}개):")
print(df.columns.tolist())

데이터 로드



로드 완료
  Shape: (844822, 33)
  기간: 2018-11-29 ~ 2026-01-09
  기업 수: 481

컬럼 (33개):
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Company', 'Dividends', 'Stock Splits', 'Sector', 'Industry', 'Daily_Return', 'Cum_Return', 'MA_5', 'MA_20', 'MA_60', 'Volatility_20d', 'Cum_Max', 'Drawdown', 'MDD', 'Vol_MA_20', 'Vol_Ratio', 'Vol_Std_20', 'Vol_Z_Score', 'Prev_Close', 'Gap', 'Gap_Pct', 'RSI_14', 'BB_Middle', 'BB_Upper', 'BB_Lower', 'BB_Width', 'Is_Extreme_Change']


---

## Section 2. 데이터 품질 검증 (QA)

In [4]:
# 2.1 결측치 현황
print("=" * 60)
print("2.1 결측치 현황")
print("=" * 60)

null_counts = df.isnull().sum()
null_pct = (null_counts / len(df) * 100).round(2)
null_summary = pd.DataFrame({'Count': null_counts, 'Percent': null_pct})
null_summary = null_summary[null_summary['Count'] > 0].sort_values('Count', ascending=False)

if len(null_summary) > 0:
    print(null_summary)
else:
    print("결측치 없음")

2.1 결측치 현황
                Count  Percent
MA_60           28380     3.36
Volatility_20d   9621     1.14
BB_Width         9140     1.08
Vol_Ratio        9140     1.08
Vol_Std_20       9140     1.08
Vol_Z_Score      9140     1.08
MA_20            9140     1.08
Vol_MA_20        9140     1.08
BB_Middle        9140     1.08
BB_Upper         9140     1.08
BB_Lower         9140     1.08
RSI_14           6254     0.74
MA_5             1925     0.23
Daily_Return      482     0.06
Gap_Pct           482     0.06
Prev_Close        482     0.06
Gap               482     0.06
Date                1     0.00
MDD                 1     0.00
Open                1     0.00
Cum_Max             1     0.00
Cum_Return          1     0.00
Company             1     0.00
Volume              1     0.00
Close               1     0.00
Low                 1     0.00
High                1     0.00
Drawdown            1     0.00


In [5]:
# 2.2 기업별 관측일 수
print("\n" + "=" * 60)
print("2.2 기업별 관측일 수")
print("=" * 60)

company_obs = df.groupby('Company').size()
print(company_obs.describe())

threshold = company_obs.max() * 0.8
partial_data = company_obs[company_obs < threshold]
print(f"\n관측일 부족 기업 (80% 미만): {len(partial_data)}개")


2.2 기업별 관측일 수
count     481.000000
mean     1756.384615
std       143.405470
min       583.000000
25%      1787.000000
50%      1787.000000
75%      1787.000000
max      1787.000000
dtype: float64

관측일 부족 기업 (80% 미만): 18개


In [6]:
# 2.3 OHLC 이상치 검증
print("\n" + "=" * 60)
print("2.3 OHLC 이상치 검증")
print("=" * 60)

anomalies = []
tolerance = 0.01

for col in ['Open', 'High', 'Low', 'Close']:
    if col in df.columns:
        invalid = (df[col] <= 0).sum()
        if invalid > 0:
            anomalies.append(f"{col} <= 0: {invalid}건")

if 'High' in df.columns and 'Low' in df.columns:
    inverted = (df['High'] < df['Low'] - tolerance).sum()
    if inverted > 0:
        anomalies.append(f"High < Low: {inverted}건")

if anomalies:
    for a in anomalies:
        print(f"  - {a}")
else:
    print("이상치 없음")


2.3 OHLC 이상치 검증
이상치 없음


In [7]:
# 2.4 거래량 0 보정
print("\n" + "=" * 60)
print("2.4 거래량 0 보정")
print("=" * 60)

volume_corrections = {
    ('BIIB', '2023-06-09'): 8067,
    ('BKR', '2019-10-18'): 2830000,
    ('CCI', '2020-07-31'): 3310000,
    ('CHD', '2019-12-31'): 1610000,
    ('CNC', '2020-02-26'): 9800000,
    ('DB', '2019-12-27'): 2530000,
    ('IBKR', '2019-10-07'): 1640000,
    ('IBKR', '2019-10-08'): 2680000,
    ('MBLY', '2022-10-27'): 5670000,
    ('NWG', '2022-08-30'): 1460,
    ('NWG', '2022-08-31'): 109130,
    ('SYM', '2021-09-22'): 108,
    ('SYM', '2021-10-12'): 72,
    ('SYM', '2022-02-17'): 138
}

df['Date_str'] = df['Date'].dt.strftime('%Y-%m-%d')
zero_before = (df['Volume'] == 0).sum()

for (company, date_str), volume in volume_corrections.items():
    mask = (df['Company'] == company) & (df['Date_str'] == date_str)
    if mask.any() and df.loc[mask, 'Volume'].values[0] == 0:
        df.loc[mask, 'Volume'] = volume

df = df.drop(columns=['Date_str'])
zero_after = (df['Volume'] == 0).sum()

print(f"거래량 0: {zero_before} -> {zero_after} (보정: {zero_before - zero_after}건)")


2.4 거래량 0 보정


거래량 0: 329 -> 317 (보정: 12건)


In [8]:
# 2.5 섹터별 분포
print("\n" + "=" * 60)
print("2.5 섹터별 기업 분포")
print("=" * 60)

if 'Sector' in df.columns:
    sector_dist = df.groupby('Sector')['Company'].nunique().sort_values(ascending=False)
    print(sector_dist)


2.5 섹터별 기업 분포
Sector
Financial Services        82
Technology                77
Healthcare                56
Industrials               56
Consumer Cyclical         45
Energy                    38
Consumer Defensive        31
Communication Services    27
Basic Materials           26
Utilities                 24
Real Estate               19
Unknown                    0
Name: Company, dtype: int64


---

## Section 3. 파생변수 계산

### 3.1 수익률 변수
- `Daily_Return_raw`: 관측/통계/ML 용도 (첫 날 NaN 유지)
- `Daily_Return_calc`: 누적수익률 계산용 (첫 날 0으로 대체)
- `Cum_Return`: 누적 수익률
- `Return_1M/3M/6M`: 롤링 모멘텀

## 수익률 변수 2종을 분리하는 이유

- `Daily_Return_raw`: **관측/통계/ML 용도**  
  - 첫 날은 정의 불가 → NaN 유지
  - 변동성(rolling std) 계산 시 **0으로 채우면 변동성 왜곡**이 생김

- `Daily_Return_calc`: **누적수익률 계산용 보조 변수**  
  - Cum_Return 계산 시 첫 값이 NaN이면 누적이 NaN으로 전파되므로 첫 날만 0으로 대체
  - 모델 입력용으로는 사용하지 않는 것을 권장

In [9]:
# 데이터 정렬
print("=" * 60)
print("Section 3. 파생변수 계산")
print("=" * 60)

df = df.sort_values(['Company', 'Date']).reset_index(drop=True)
print("데이터 정렬 완료")

Section 3. 파생변수 계산
데이터 정렬 완료


In [10]:
# 3.1 수익률 변수
print("\n[3.1] 수익률 변수 계산")

grouped_close = df.groupby('Company')['Close']

# Daily Return (NaN 유지 - 관측/통계/ML용)
df['Daily_Return_raw'] = grouped_close.pct_change()
df['Daily_Return_raw'] = df['Daily_Return_raw'].replace([np.inf, -np.inf], np.nan)

# Daily Return (계산용 - 누적수익률용, 첫날 0)
df['Daily_Return_calc'] = df['Daily_Return_raw'].fillna(0)

# Cumulative Return
df['Cum_Return'] = df.groupby('Company')['Daily_Return_calc'].transform(
    lambda x: (1 + x).cumprod() - 1
)

# Rolling Returns (모멘텀)
df['Return_1M'] = grouped_close.pct_change(periods=20)
df['Return_3M'] = grouped_close.pct_change(periods=60)
df['Return_6M'] = grouped_close.pct_change(periods=120)

print("  - Daily_Return_raw, Daily_Return_calc 완료")
print("  - Cum_Return 완료")
print("  - Return_1M/3M/6M 완료")


[3.1] 수익률 변수 계산


  - Daily_Return_raw, Daily_Return_calc 완료
  - Cum_Return 완료
  - Return_1M/3M/6M 완료


## 이동평균(MA)에서 min_periods를 window로 주는 이유
초기 구간(예: MA_60의 첫 59일)은 '60일 평균'이라는 의미가 성립하지 않음.
따라서 `min_periods=window`로 두고, 최종 모델링 시 dropna로 정리하는 방식이 가장 정석

In [11]:
# 3.2 이동평균
print("\n[3.2] 이동평균 계산")

df['MA_5'] = grouped_close.transform(lambda x: x.rolling(window=5, min_periods=5).mean())
df['MA_20'] = grouped_close.transform(lambda x: x.rolling(window=20, min_periods=20).mean())
df['MA_60'] = grouped_close.transform(lambda x: x.rolling(window=60, min_periods=60).mean())

print("  - MA_5, MA_20, MA_60 완료")


[3.2] 이동평균 계산
  - MA_5, MA_20, MA_60 완료


## 변동성(Volatility)
- 일별 수익률의 표준편차를 rolling으로 계산한 뒤,
- 연환산을 위해 `sqrt(252)`를 곱합니다.
- 여기서는 **Daily_Return_raw(NaN 유지)** 로 계산하는 것을 권장

In [12]:
# 3.3 변동성 (연환산)
print("\n[3.3] 변동성 계산")

df['Volatility_20d'] = df.groupby('Company')['Daily_Return_raw'].transform(
    lambda x: x.rolling(window=20, min_periods=20).std() * np.sqrt(252)
)

print("  - Volatility_20d (연환산) 완료")


[3.3] 변동성 계산


  - Volatility_20d (연환산) 완료


## Drawdown / MDD (Rolling Window)
- Drawdown: (현재가 / 롤링 최고가) - 1
- MDD: 해당 기간 동안 drawdown의 최솟값(가장 큰 낙폭)
- `min_periods=1`은 초기 구간을 0 근처로 만들며 '1년 기준' 의미가 약해질 수 있어,
  252일 기준 지표라면 `min_periods=252`를 사용

In [13]:
# 3.4 리스크 지표 (Drawdown, MDD)
print("\n[3.4] 리스크 지표 계산")

window_year = 252
window_short = 60

# 1년 기준 Rolling Max & Drawdown
roll_max_year = (
    df.groupby('Company')['Close']
      .rolling(window=window_year, min_periods=window_year)
      .max()
      .reset_index(level=0, drop=True)
)
df['Drawdown'] = df['Close'] / roll_max_year - 1.0

# MDD (Maximum Drawdown)
df['MDD'] = (
    df['Drawdown'].groupby(df['Company'])
    .rolling(window=window_year, min_periods=window_year)
    .min()
    .reset_index(level=0, drop=True)
)

# 60일 기준 Drawdown
roll_max_short = (
    df.groupby('Company')['Close']
      .rolling(window=window_short, min_periods=window_short)
      .max()
      .reset_index(level=0, drop=True)
)
df['DD_Short'] = df['Close'] / roll_max_short - 1.0

print("  - Drawdown (1Y), MDD (1Y), DD_Short (60D) 완료")


[3.4] 리스크 지표 계산


  - Drawdown (1Y), MDD (1Y), DD_Short (60D) 완료


## 거래량(Volume) 파생변수
- Vol_MA_20: 20일 평균 거래량
- Vol_Ratio: 당일 / 20일 평균 (분모 0 방지 필요)
- Vol_Z_Score: (당일 - 평균) / 표준편차 (std=0 방지 필요)

추가 전처리(ML에 특히 유용):
- log1p(Volume): 치우친 분포를 완화
- winsorize(상하 1%): 이상치를 제거가 아니라 '캡핑'해서 안정화
- 권장 순서: **log1p → winsorize**

In [14]:
# 3.5 거래량 지표
print("\n[3.5] 거래량 지표 계산")

grouped_vol = df.groupby('Company')['Volume']
window = 20

# Volume MA & Ratio
df['Vol_MA_20'] = grouped_vol.transform(lambda x: x.rolling(window=window, min_periods=window).mean())
df['Vol_Ratio'] = df['Volume'] / df['Vol_MA_20'].replace(0, np.nan)

# Volume Z-Score
df['Vol_Std_20'] = grouped_vol.transform(lambda x: x.rolling(window=window, min_periods=window).std())
df['Vol_Z_Score'] = (df['Volume'] - df['Vol_MA_20']) / df['Vol_Std_20'].replace(0, np.nan)

# Log Volume + Winsorize
df['Log_Volume'] = np.log1p(df['Volume'])

def winsorize_series(s, lower=0.01, upper=0.99):
    lo, hi = s.quantile(lower), s.quantile(upper)
    return s.clip(lower=lo, upper=hi)

df['Log_Volume_W'] = df.groupby('Company')['Log_Volume'].transform(winsorize_series)

# inf 처리
df[['Vol_Ratio', 'Vol_Z_Score']] = df[['Vol_Ratio', 'Vol_Z_Score']].replace([np.inf, -np.inf], np.nan)

print("  - Vol_MA_20, Vol_Ratio, Vol_Z_Score 완료")
print("  - Log_Volume, Log_Volume_W (winsorize 1%) 완료")


[3.5] 거래량 지표 계산


  - Vol_MA_20, Vol_Ratio, Vol_Z_Score 완료
  - Log_Volume, Log_Volume_W (winsorize 1%) 완료


In [15]:
# 3.6 기술지표 (RSI, Bollinger Bands)
print("\n[3.6] 기술지표 계산")

# RSI
def calculate_rsi_wilder(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = (-delta).clip(lower=0)

    # Wilder's Smoothing (EMA 방식)
    avg_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()

    rs = avg_gain / avg_loss.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))

    # loss가 0이면 상승만 한 케이스 -> RSI=100으로 두고 싶다면 아래를 적용
    rsi = rsi.where(~(avg_loss == 0), 100)
    # gain이 0이면 하락만 한 케이스 -> RSI=0
    rsi = rsi.where(~(avg_gain == 0), 0)

    return rsi

df['RSI_14'] = df.groupby('Company')['Close'].transform(lambda x: calculate_rsi_wilder(x, 14))

# Bollinger Bands
df['BB_Middle'] = grouped_close.transform(lambda x: x.rolling(20, min_periods=20).mean())
bb_std = grouped_close.transform(lambda x: x.rolling(20, min_periods=20).std())
df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle'].replace(0, np.nan)

print("  - RSI_14 완료")
print("  - BB_Upper, BB_Middle, BB_Lower, BB_Width 완료")


[3.6] 기술지표 계산


  - RSI_14 완료
  - BB_Upper, BB_Middle, BB_Lower, BB_Width 완료


In [16]:
# 파생변수 계산 결과
print("\n" + "=" * 60)
print("파생변수 계산 결과")
print("=" * 60)

print(f"Shape: {df.shape}")
print(f"컬럼 수: {len(df.columns)}")
print(f"\n전체 컬럼:\n{df.columns.tolist()}")


파생변수 계산 결과
Shape: (844822, 41)
컬럼 수: 41

전체 컬럼:
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Company', 'Dividends', 'Stock Splits', 'Sector', 'Industry', 'Daily_Return', 'Cum_Return', 'MA_5', 'MA_20', 'MA_60', 'Volatility_20d', 'Cum_Max', 'Drawdown', 'MDD', 'Vol_MA_20', 'Vol_Ratio', 'Vol_Std_20', 'Vol_Z_Score', 'Prev_Close', 'Gap', 'Gap_Pct', 'RSI_14', 'BB_Middle', 'BB_Upper', 'BB_Lower', 'BB_Width', 'Is_Extreme_Change', 'Daily_Return_raw', 'Daily_Return_calc', 'Return_1M', 'Return_3M', 'Return_6M', 'DD_Short', 'Log_Volume', 'Log_Volume_W']


---

## Section 4. 결측치 처리 및 저장

In [17]:
# 4.1 결측치 현황
print("=" * 60)
print("4.1 결측치 현황")
print("=" * 60)

null_counts = df.isnull().sum()
null_pct = (null_counts / len(df) * 100).round(2)
null_summary = pd.DataFrame({'Count': null_counts, 'Percent': null_pct})
null_summary = null_summary[null_summary['Count'] > 0].sort_values('Count', ascending=False)

print(null_summary)
print(f"\n총 결측 컬럼: {len(null_summary)}개")

4.1 결측치 현황
                   Count  Percent
MDD               241463    28.58
Drawdown          120732    14.29
Return_6M          57721     6.83
Return_3M          28861     3.42
DD_Short           28380     3.36
MA_60              28380     3.36
Volatility_20d      9621     1.14
Return_1M           9621     1.14
BB_Width            9140     1.08
Vol_Z_Score         9140     1.08
Vol_Std_20          9140     1.08
BB_Lower            9140     1.08
Vol_MA_20           9140     1.08
Vol_Ratio           9140     1.08
BB_Upper            9140     1.08
MA_20               9140     1.08
BB_Middle           9140     1.08
RSI_14              6735     0.80
MA_5                1925     0.23
Daily_Return_raw     482     0.06
Daily_Return         482     0.06
Prev_Close           482     0.06
Gap                  482     0.06
Gap_Pct              482     0.06
Log_Volume             1     0.00
Date                   1     0.00
Open                   1     0.00
Cum_Max                1     0.00
Cum

In [18]:
# 4.2 컬럼 순서 정리
print("\n" + "=" * 60)
print("4.2 컬럼 순서 정리")
print("=" * 60)

column_order = [
    # 기본 정보
    'Date', 'Company', 'Sector', 'Industry',
    # OHLCV
    'Open', 'High', 'Low', 'Close', 'Volume',
    # 수익률
    'Daily_Return_raw', 'Daily_Return_calc', 'Cum_Return',
    'Return_1M', 'Return_3M', 'Return_6M',
    # 이동평균
    'MA_5', 'MA_20', 'MA_60',
    # 변동성
    'Volatility_20d',
    # 리스크
    'Drawdown', 'MDD', 'DD_Short',
    # 거래량 지표
    'Vol_MA_20', 'Vol_Ratio', 'Vol_Std_20', 'Vol_Z_Score',
    'Log_Volume', 'Log_Volume_W',
    # 기술지표
    'RSI_14', 'BB_Upper', 'BB_Middle', 'BB_Lower', 'BB_Width',
]

# 존재하는 컬럼만 선택 + 나머지 추가
final_cols = [col for col in column_order if col in df.columns]
remaining_cols = [col for col in df.columns if col not in final_cols]
final_cols.extend(remaining_cols)

df = df[final_cols].copy()
print(f"컬럼 정리 완료: {len(df.columns)}개")


4.2 컬럼 순서 정리


컬럼 정리 완료: 41개


In [19]:
# 4.3 전체 데이터 저장 (NaN 포함)
print("\n" + "=" * 60)
print("4.3 전체 데이터 저장 (NaN 포함)")
print("=" * 60)

output_full = DATA_DIR / 'stock_features_full.csv'
df.to_csv(output_full, index=False, date_format='%Y-%m-%d', float_format='%.10g')

print(f"저장 완료: {output_full.name}")
print(f"Shape: {df.shape}")
print(f"파일 크기: {output_full.stat().st_size / (1024*1024):.2f} MB")


4.3 전체 데이터 저장 (NaN 포함)


저장 완료: stock_features_full.csv
Shape: (844822, 41)
파일 크기: 385.14 MB


In [20]:
# 4.4 Clean 데이터 생성 (dropna 적용)
print("\n" + "=" * 60)
print("4.4 Clean 데이터 생성 (dropna 적용)")
print("=" * 60)

# 분석/모델링에 사용할 핵심 컬럼 기준으로 dropna
key_columns = [
    'Daily_Return_raw','Cum_Return',
    'Return_1M','Return_3M','Return_6M',
    'MA_5','MA_20','MA_60',
    'Volatility_20d',
    'Drawdown','MDD','DD_Short',
    'Vol_MA_20','Vol_Ratio','Vol_Z_Score','Log_Volume_W',
    'RSI_14','BB_Width'
]

# 존재하는 컬럼만 선택
key_columns = [col for col in key_columns if col in df.columns]

print(f"dropna 기준 컬럼: {key_columns}")

df_clean = df.dropna(subset=key_columns).reset_index(drop=True)

print(f"\n원본: {len(df):,} rows")
print(f"Clean: {len(df_clean):,} rows")
print(f"제거: {len(df) - len(df_clean):,} rows ({(len(df) - len(df_clean)) / len(df) * 100:.2f}%)")


4.4 Clean 데이터 생성 (dropna 적용)
dropna 기준 컬럼: ['Daily_Return_raw', 'Cum_Return', 'Return_1M', 'Return_3M', 'Return_6M', 'MA_5', 'MA_20', 'MA_60', 'Volatility_20d', 'Drawdown', 'MDD', 'DD_Short', 'Vol_MA_20', 'Vol_Ratio', 'Vol_Z_Score', 'Log_Volume_W', 'RSI_14', 'BB_Width']

원본: 844,822 rows
Clean: 603,359 rows
제거: 241,463 rows (28.58%)


In [21]:
# 4.5 Clean 데이터 검증
print("\n" + "=" * 60)
print("4.5 Clean 데이터 검증")
print("=" * 60)

print(f"Shape: {df_clean.shape}")
print(f"기업 수: {df_clean['Company'].nunique()}")
print(f"기간: {df_clean['Date'].min().date()} ~ {df_clean['Date'].max().date()}")

# 기업별 관측일 수
clean_obs = df_clean.groupby('Company').size()
print(f"\n기업별 관측일 수:")
print(clean_obs.describe())

# 결측치 확인
null_check = df_clean[key_columns].isnull().sum()
print(f"\n핵심 컬럼 결측치: {null_check.sum()} (모두 0이어야 함)")


4.5 Clean 데이터 검증
Shape: (603359, 41)
기업 수: 481
기간: 2020-11-27 ~ 2026-01-09

기업별 관측일 수:
count     481.000000
mean     1254.384615
std       143.405470
min        81.000000
25%      1285.000000
50%      1285.000000
75%      1285.000000
max      1285.000000
dtype: float64

핵심 컬럼 결측치: 0 (모두 0이어야 함)


In [22]:
# 4.6 Clean 데이터 저장
print("\n" + "=" * 60)
print("4.6 Clean 데이터 저장")
print("=" * 60)

output_clean = DATA_DIR / 'stock_features_clean.csv'
df_clean.to_csv(output_clean, index=False, date_format='%Y-%m-%d', float_format='%.10g')

print(f"저장 완료: {output_clean.name}")
print(f"Shape: {df_clean.shape}")
print(f"파일 크기: {output_clean.stat().st_size / (1024*1024):.2f} MB")


4.6 Clean 데이터 저장


저장 완료: stock_features_clean.csv
Shape: (603359, 41)
파일 크기: 280.25 MB


---

## Section 5. 최종 요약

In [23]:
# 최종 요약 리포트
print("=" * 60)
print("최종 요약 리포트")
print("=" * 60)

print("\n[생성된 파일]")
print(f"  1. stock_features_full.csv  - 전체 데이터 (NaN 포함)")
print(f"     Shape: {df.shape}, Size: {output_full.stat().st_size / (1024*1024):.2f} MB")
print(f"  2. stock_features_clean.csv - Clean 데이터 (dropna 적용)")
print(f"     Shape: {df_clean.shape}, Size: {output_clean.stat().st_size / (1024*1024):.2f} MB")

print("\n[데이터 현황 - Clean]")
print(f"  - 기업 수: {df_clean['Company'].nunique()}")
print(f"  - 기간: {df_clean['Date'].min().date()} ~ {df_clean['Date'].max().date()}")
print(f"  - 총 행 수: {len(df_clean):,}")
print(f"  - 컬럼 수: {len(df_clean.columns)}")

print("\n[섹터 분포 - Clean]")
if 'Sector' in df_clean.columns:
    print(df_clean.groupby('Sector')['Company'].nunique().sort_values(ascending=False))

print("\n[주요 지표 통계 - Clean]")
stats_cols = ['Daily_Return_raw', 'MDD', 'Volatility_20d', 'RSI_14']
stats_cols = [col for col in stats_cols if col in df_clean.columns]
print(df_clean[stats_cols].describe())

print("\n" + "=" * 60)
print("모든 작업 완료")
print("=" * 60)

최종 요약 리포트

[생성된 파일]
  1. stock_features_full.csv  - 전체 데이터 (NaN 포함)
     Shape: (844822, 41), Size: 385.14 MB
  2. stock_features_clean.csv - Clean 데이터 (dropna 적용)
     Shape: (603359, 41), Size: 280.25 MB

[데이터 현황 - Clean]
  - 기업 수: 481
  - 기간: 2020-11-27 ~ 2026-01-09
  - 총 행 수: 603,359
  - 컬럼 수: 41

[섹터 분포 - Clean]
Sector
Financial Services        82
Technology                77
Healthcare                56
Industrials               56
Consumer Cyclical         45
Energy                    38
Consumer Defensive        31
Communication Services    27
Basic Materials           26
Utilities                 24
Real Estate               19
Name: Company, dtype: int64

[주요 지표 통계 - Clean]
       Daily_Return_raw            MDD  Volatility_20d         RSI_14
count     603359.000000  603359.000000   603359.000000  603359.000000
mean           0.000667      -0.303729        0.292359      52.354884
std            0.020652       0.146437        0.153516      12.107051
min           -0.440438    